In [2]:
import pandas as pd
import numpy as np


In [3]:
# Load all three datasets
final_data = pd.read_csv('../data/processed/final_data.csv')
final_total = pd.read_csv('../data/processed/final_total_data.csv')
economic_damage = pd.read_csv('../data/raw/total_count/economic-damages-from-disasters/economic_damage.csv')

print("Dataset shapes:")
print(f"final_data: {final_data.shape}")
print(f"final_total: {final_total.shape}")
print(f"economic_damage: {economic_damage.shape}")

print("\nfinal_data columns:", list(final_data.columns))
print("\nfinal_total columns:", list(final_total.columns))
print("\neconomic_damage columns:", list(economic_damage.columns))


Dataset shapes:
final_data: (2176, 11)
final_total: (2440, 10)
economic_damage: (5615, 7)

final_data columns: ['Unnamed: 0.1', 'Unnamed: 0', 'country', 'period', 'flood_impact', 'drought_impact', 'storms_impact', 'extreme_temp_impact', 'climate_impact_index', 'hazard_count', 'impact_rebased']

final_total columns: ['country', 'period', 'extreme_temp_total_affected', 'extreme_temp_total_deaths', 'drought_total_affected', 'drought_total_deaths', 'flood_total_affected', 'flood_total_deaths', 'storm_total_affected', 'storm_total_deaths']

economic_damage columns: ['Entity', 'Code', 'Year', 'drought', 'flood', 'storm', 'extreme_temp']


In [4]:
# Process economic_damage.csv: Convert Year → 5-year period and aggregate
economic_damage['period'] = (economic_damage['Year'] // 5) * 5
economic_damage.rename(columns={'Entity': 'country'}, inplace=True)

# Keep only the hazard columns we need
hazard_cols = ['drought', 'flood', 'storm', 'extreme_temp']
economic_cols = ['country', 'period'] + hazard_cols

# Aggregate economic damage to country-period using mean across hazards
economic_agg = (
    economic_damage[economic_cols]
    .groupby(['country', 'period'])[hazard_cols]
    .mean()
    .reset_index()
)

# Compute mean economic damage across all hazards for each country-period
economic_agg['economic_damage_pct_gdp'] = economic_agg[hazard_cols].mean(axis=1)

# Keep only country, period, and the aggregated economic damage
economic_processed = economic_agg[['country', 'period', 'economic_damage_pct_gdp']].copy()

print("Economic damage processed:")
print(economic_processed.head(10))
print(f"\nShape: {economic_processed.shape}")


Economic damage processed:
       country  period  economic_damage_pct_gdp
0  Afghanistan    2000                 0.000286
1  Afghanistan    2005                 0.018129
2  Afghanistan    2010                 0.200113
3  Afghanistan    2015                 0.000000
4  Afghanistan    2020                 0.000000
5      Albania    1980                 0.000000
6      Albania    1985                 0.000000
7      Albania    1990                 0.536666
8      Albania    1995                 0.000000
9      Albania    2000                 0.064989

Shape: (1796, 3)


In [5]:
# Process final_total.csv: Compute total_deaths and total_affected
total_deaths_cols = [
    'extreme_temp_total_deaths',
    'drought_total_deaths',
    'flood_total_deaths',
    'storm_total_deaths'
]

total_affected_cols = [
    'extreme_temp_total_affected',
    'drought_total_affected',
    'flood_total_affected',
    'storm_total_affected'
]

final_total['total_deaths'] = final_total[total_deaths_cols].sum(axis=1)
final_total['total_affected'] = final_total[total_affected_cols].sum(axis=1)

# Keep only country, period, total_deaths, total_affected
total_processed = final_total[['country', 'period', 'total_deaths', 'total_affected']].copy()

print("Total deaths and affected processed:")
print(total_processed.head(10))
print(f"\nShape: {total_processed.shape}")


Total deaths and affected processed:
       country  period  total_deaths  total_affected
0  Afghanistan    1955          51.0             0.0
1  Afghanistan    1960         107.0             0.0
2  Afghanistan    1965           0.0         48000.0
3  Afghanistan    1970         150.0        250000.0
4  Afghanistan    1975         171.0        351684.0
5  Afghanistan    1980           0.0         30000.0
6  Afghanistan    1985           0.0        161000.0
7  Afghanistan    1990        2027.0        143600.0
8  Afghanistan    1995         197.0         38905.0
9  Afghanistan    2000         597.0       2793798.0

Shape: (2440, 4)


In [6]:
# Clean final_data: Remove Unnamed column if exists and keep only needed columns
if 'Unnamed: 0' in final_data.columns:
    final_data = final_data.drop(columns=['Unnamed: 0'])

# Keep impact columns and metrics
impact_cols = ['country', 'period', 
               'flood_impact', 'drought_impact', 'storms_impact', 'extreme_temp_impact',
               'climate_impact_index', 'hazard_count', 'impact_rebased']

final_data_processed = final_data[impact_cols].copy()

print("Final data processed:")
print(final_data_processed.head(10))
print(f"\nShape: {final_data_processed.shape}")


Final data processed:
       country  period  flood_impact  drought_impact  storms_impact  \
0  Afghanistan    1965           NaN       -0.069536            NaN   
1  Afghanistan    1970           NaN       -1.661444            NaN   
2  Afghanistan    1975      1.457512             NaN            NaN   
3  Afghanistan    1990      4.003976             NaN      -1.466066   
4  Afghanistan    1995     -0.578217             NaN            NaN   
5  Afghanistan    2000     -0.712863        0.569529            NaN   
6  Afghanistan    2005      0.471154        0.527329       0.196208   
7  Afghanistan    2010      0.541070        0.615053      -1.400176   
8  Afghanistan    2015     -0.342157        0.906295      -0.349956   
9  Afghanistan    2020      1.066724        0.724497      -0.581605   

   extreme_temp_impact  climate_impact_index  hazard_count  impact_rebased  
0                  NaN             -0.069536             1        2.500123  
1                  NaN             -1.6614

In [8]:
# Merge all datasets on (country, period)
analysis_dataset = final_data_processed.merge(
    total_processed, 
    on=['country', 'period'], 
    how='left'
).merge(
    economic_processed,
    on=['country', 'period'],
    how='left'
)

print("After merging:")
print(f"Shape: {analysis_dataset.shape}")
print(f"Columns: {list(analysis_dataset.columns)}")
print("\nFirst few rows:")
print(analysis_dataset.head())


After merging:
Shape: (2176, 12)
Columns: ['country', 'period', 'flood_impact', 'drought_impact', 'storms_impact', 'extreme_temp_impact', 'climate_impact_index', 'hazard_count', 'impact_rebased', 'total_deaths', 'total_affected', 'economic_damage_pct_gdp']

First few rows:
       country  period  flood_impact  drought_impact  storms_impact  \
0  Afghanistan    1965           NaN       -0.069536            NaN   
1  Afghanistan    1970           NaN       -1.661444            NaN   
2  Afghanistan    1975      1.457512             NaN            NaN   
3  Afghanistan    1990      4.003976             NaN      -1.466066   
4  Afghanistan    1995     -0.578217             NaN            NaN   

   extreme_temp_impact  climate_impact_index  hazard_count  impact_rebased  \
0                  NaN             -0.069536             1        2.500123   
1                  NaN             -1.661444             1        0.908214   
2                  NaN              1.457512             1       

In [9]:
# Compute resilience metrics
# Handle division by zero: if total_affected is 0, resilience_rate should be 0 or NaN
analysis_dataset['resilience_rate'] = np.where(
    analysis_dataset['total_affected'] > 0,
    analysis_dataset['total_deaths'] / analysis_dataset['total_affected'],
    0
)

analysis_dataset['resilience_per_100k'] = analysis_dataset['resilience_rate'] * 100000

# Fill NaN values in economic_damage_pct_gdp with 0 (no economic damage recorded)
analysis_dataset['economic_damage_pct_gdp'] = analysis_dataset['economic_damage_pct_gdp'].fillna(0)

print("Resilience metrics computed:")
print(analysis_dataset[['country', 'period', 'total_deaths', 'total_affected', 
                         'resilience_rate', 'resilience_per_100k', 'economic_damage_pct_gdp']].head(10))


Resilience metrics computed:
       country  period  total_deaths  total_affected  resilience_rate  \
0  Afghanistan    1965           0.0         48000.0         0.000000   
1  Afghanistan    1970         150.0        250000.0         0.000600   
2  Afghanistan    1975         171.0        351684.0         0.000486   
3  Afghanistan    1990        2027.0        143600.0         0.014116   
4  Afghanistan    1995         197.0         38905.0         0.005064   
5  Afghanistan    2000         597.0       2793798.0         0.000214   
6  Afghanistan    2005        2599.0       2518584.0         0.001032   
7  Afghanistan    2010        1100.0       2018185.0         0.000545   
8  Afghanistan    2015         466.0      13648829.0         0.000034   
9  Afghanistan    2020        2979.0      12120852.0         0.000246   

   resilience_per_100k  economic_damage_pct_gdp  
0             0.000000                 0.000000  
1            60.000000                 0.000000  
2            48.6

In [10]:
# Create final clean dataframe with required columns in specified order
final_columns = [
    'country', 'period',
    'flood_impact', 'drought_impact', 'storms_impact', 'extreme_temp_impact',
    'climate_impact_index', 'impact_rebased', 'hazard_count',
    'total_deaths', 'total_affected',
    'resilience_rate', 'resilience_per_100k',
    'economic_damage_pct_gdp'
]

analysis_data_set = analysis_dataset[final_columns].copy()

# Sort by country and period
analysis_data_set = analysis_data_set.sort_values(['country', 'period']).reset_index(drop=True)

print("Final analysis dataset:")
print(analysis_data_set.head(15))
print(f"\nShape: {analysis_data_set.shape}")
print(f"\nColumns: {list(analysis_data_set.columns)}")
print(f"\nData summary:")
print(analysis_data_set.describe())


Final analysis dataset:
        country  period  flood_impact  drought_impact  storms_impact  \
0   Afghanistan    1965           NaN       -0.069536            NaN   
1   Afghanistan    1970           NaN       -1.661444            NaN   
2   Afghanistan    1975      1.457512             NaN            NaN   
3   Afghanistan    1990      4.003976             NaN      -1.466066   
4   Afghanistan    1995     -0.578217             NaN            NaN   
5   Afghanistan    2000     -0.712863        0.569529            NaN   
6   Afghanistan    2005      0.471154        0.527329       0.196208   
7   Afghanistan    2010      0.541070        0.615053      -1.400176   
8   Afghanistan    2015     -0.342157        0.906295      -0.349956   
9   Afghanistan    2020      1.066724        0.724497      -0.581605   
10       Africa    1900           NaN       -0.325274            NaN   
11       Africa    1905           NaN       -1.661444            NaN   
12       Africa    1910           NaN   

In [11]:
# Save final analysis dataset
analysis_data_set.to_csv('../data/processed/analysis_data_set.csv', index=False)

print("Analysis dataset saved to: ../data/processed/analysis_data_set.csv")
print(f"\nFinal dataset shape: {analysis_data_set.shape}")
print(f"\nSample data:")
print(analysis_data_set.head(20))


Analysis dataset saved to: ../data/processed/analysis_data_set.csv

Final dataset shape: (2176, 14)

Sample data:
        country  period  flood_impact  drought_impact  storms_impact  \
0   Afghanistan    1965           NaN       -0.069536            NaN   
1   Afghanistan    1970           NaN       -1.661444            NaN   
2   Afghanistan    1975      1.457512             NaN            NaN   
3   Afghanistan    1990      4.003976             NaN      -1.466066   
4   Afghanistan    1995     -0.578217             NaN            NaN   
5   Afghanistan    2000     -0.712863        0.569529            NaN   
6   Afghanistan    2005      0.471154        0.527329       0.196208   
7   Afghanistan    2010      0.541070        0.615053      -1.400176   
8   Afghanistan    2015     -0.342157        0.906295      -0.349956   
9   Afghanistan    2020      1.066724        0.724497      -0.581605   
10       Africa    1900           NaN       -0.325274            NaN   
11       Africa    190